In [1]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset
import numpy as np
import pandas as pd
import evaluate
import wandb

DATA_DIR = "/media/hdddisk/bert-classify-smsspam-data"

/media/hdddisk/anaconda3/envs/nlp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/hdddisk/anaconda3/envs/nlp/lib/python3.11/site-packages/transformers/utils/hub.py:128: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2025-02-06 09:52:28.852449: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-06 09:52:28.871672: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the foll

In [2]:
df = pd.read_csv("./data/smsspam.csv")

# split into train and test
train_df = df.sample(frac=0.8, random_state=42)
test_df = df.drop(train_df.index)

train_df.to_csv("./data/smsspam-train.csv", index=False)
test_df.to_csv("./data/smsspam-test.csv", index=False)

print("Train size:", len(train_df), "Test size:", len(test_df))

Train size: 4459 Test size: 1115


In [3]:
# Load the dataset
dataset = load_dataset('csv', data_files={'train': './data/smsspam-train.csv', 'test': './data/smsspam-test.csv'})

# Load the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def tokenize_function(examples):
    return tokenizer(examples['text'], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare the dataset for PyTorch
tokenized_datasets.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

# Load the BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

Generating train split: 4459 examples [00:00, 673717.63 examples/s]
Generating test split: 1115 examples [00:00, 389007.57 examples/s]
Map: 100%|██████████| 1115/1115 [00:00<00:00, 4102.74 examples/s]
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
training_args = TrainingArguments(
    output_dir=f'{DATA_DIR}/results',  # Output directory
    num_train_epochs=1,                # Number of training epochs
    per_device_train_batch_size=8,     # Batch size for training
    per_device_eval_batch_size=8,      # Batch size for evaluation
    learning_rate=1e-5,                # Learning rate
    warmup_steps=100,                   # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,                 # Strength of weight decay
    logging_dir=f'{DATA_DIR}/logs',    # Directory for storing logs
    logging_steps=10,
    eval_strategy="steps",             # Evaluate every epoch = "epoch"
    eval_steps=50,                     # Number of steps between evaluations
    save_strategy="steps",             # Save model every epoch
    save_steps=50,
    load_best_model_at_end=True,       # Load the best model at the end of training
    save_total_limit=2,                # Limit the total number of saved models
    metric_for_best_model="accuracy",  # Use accuracy to determine the best model
)

# Define the metric for evaluation
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
# Initialize the Trainer
trainer = Trainer(
    model=model,                         # The pre-trained model
    args=training_args,                  # Training arguments
    train_dataset=tokenized_datasets['train'],  # Training dataset
    eval_dataset=tokenized_datasets['test'],    # Evaluation dataset
    compute_metrics=compute_metrics,     # Function to compute metrics
)

In [6]:
# Train the model
wandb.init()
trainer.train()

wandb: Currently logged in as: yishenggong9437 (gongyisheng) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Step,Training Loss,Validation Loss,Accuracy
50,0.414000,0.396844,0.860987
100,0.102000,0.102034,0.977578
150,0.047100,0.072170,0.980269
200,0.005300,0.073822,0.980269
250,0.003200,0.090571,0.979372
300,0.172400,0.094470,0.982960
350,0.048700,0.089385,0.984753
400,0.073300,0.102245,0.981166
450,0.003900,0.079671,0.985650
500,0.083900,0.080319,0.984753


TrainOutput(global_step=558, training_loss=0.09876843937255415, metrics={'train_runtime': 493.5967, 'train_samples_per_second': 9.034, 'train_steps_per_second': 1.13, 'total_flos': 1173212195850240.0, 'train_loss': 0.09876843937255415, 'epoch': 1.0})

In [10]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")

Evaluation results: {'eval_loss': 0.0827006921172142, 'eval_accuracy': 0.9856502242152466, 'eval_runtime': 19.1159, 'eval_samples_per_second': 58.328, 'eval_steps_per_second': 7.324, 'epoch': 10.0}


In [11]:
# Save the model
trainer.save_model('./final_model')
tokenizer.save_pretrained('./final_model')

print("Model training and evaluation complete. Model saved to './final_model'.")

Model training and evaluation complete. Model saved to './final_model'.
